##Credit Card Fraud Detection
##### With a rise in online transactions credit card fraud has become a major concern.Building a model to distinguish between normal and fraudulent transaction can help us to minimize business loss.

###About the dataset
#### This dataset is obtained from kaggle.Due to confidentiality issues the original dataset cannot be obtained and the feautures V1 to V28 are obtained after PCA

#Importing the Libraries

In [67]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression #Logistic Regression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [68]:
#loading the data to a pandas dataframe
credit_card=pd.read_csv("creditcard.csv")

In [69]:
#prit first 3 rows of the dataset
credit_card.head(3)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0


In [70]:
#Check missing values
credit_card.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [71]:
#Ratio of legit and fraudulent transaction
credit_card['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

#### A highly imbalanced dataset with only 0.17% being fraudulent transaction
### 0-->Normal
### 1-->Fraudulent

In [72]:
#separate normal and fraudulent transaction
normal=credit_card[credit_card['Class']==0]
fraud=credit_card[credit_card['Class']==1]

In [73]:
print(normal.shape)
print(fraud.shape)

(284315, 31)
(492, 31)


In [74]:
normal['Amount'].describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [75]:
fraud['Amount'].describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [76]:
#groupby class and compare the mean values
credit_card.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


###Under sampling
#### a technique that is used to balance uneven datasets by reducing the size of the majority class(here class=0) and keeping the size of minority class as it is.

In [77]:
#selecting 492 random samples
legit=normal.sample(n=492,random_state=42)

In [78]:
credit_card_data=pd.concat([legit,fraud],axis=0)

In [79]:
credit_card_data.tail(2)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
281144,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1
281674,170348.0,1.991976,0.158476,-2.583441,0.408670,1.151147,-0.096695,0.223050,-0.068384,0.577829,...,-0.164350,-0.295135,-0.072173,-0.450261,0.313267,-0.289617,0.002988,-0.015309,42.53,1


In [80]:
credit_card_data['Class'].value_counts()

0    492
1    492
Name: Class, dtype: int64

In [81]:
credit_card_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,95052.758130,0.153312,0.009649,-0.038029,-0.027323,0.061966,-0.053962,0.013795,0.014911,0.037348,...,0.015030,0.014059,-0.020781,0.013223,-0.007257,0.024646,-0.027696,0.011070,-0.002305,80.348354
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [83]:
#Split the independent and dependent variables
x=credit_card_data.drop(columns='Class',axis=1)
y=credit_card_data['Class']

#Train Test Split

In [84]:
#Train Test Split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42,stratify=y)

In [85]:
print(x_train.shape,x_test.shape)

(787, 30) (197, 30)


##Logistic Regression

In [86]:
#Model training-->Logistic Regression
lr=LogisticRegression()
lr.fit(x_train,y_train)

LogisticRegression()

###Model Evaluation

In [87]:
#accuracy on training data
x_train_pred=lr.predict(x_train)
train_accuracy=accuracy_score(x_train_pred,y_train)
print("training data accuracy is ",train_accuracy)

training data accuracy is  0.9339263024142312


In [88]:
#accuracy on test data
x_test_pred=lr.predict(x_test)
test_accuracy=accuracy_score(x_test_pred,y_test)
print("test data accuracy is ", test_accuracy)

test data accuracy is  0.949238578680203


In [89]:
#f1 score on training data
train_f1_score=f1_score(x_train_pred,y_train)
print("training data f1 score is ",train_f1_score)

training data f1 score is  0.9322916666666666


In [90]:
#f1 score on test data
test_f1_score=f1_score(x_test_pred,y_test)
print("test data accuracy is ", test_f1_score)

test data accuracy is  0.9489795918367347
